<a href="https://colab.research.google.com/github/muhaiminsk/vr-activity-classification/blob/main/VR_Activity_classf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [4]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00


In [7]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import tensorflow as tf
from datetime import datetime
import kerastuner as kt

# Check if GPU is available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"GPU available: {gpus[0]}")
else:
    print("No GPU available. Using CPU.")

# Upload the training dataset
print("Upload the training dataset:")
uploaded_training = files.upload()

# Read the training dataset
df_train = pd.read_csv(io.BytesIO(list(uploaded_training.values())[0]))

# Select specified columns for training
required_columns = ['Time', 'No. of Packets', 'No. of Bits',
                    'Avg. Packet Length (Bytes)', 'Avg. Inter-packet Arrival Time', 'Activity']
for col in required_columns:
    if col not in df_train.columns:
        raise ValueError(f"Missing column in training dataset: {col}")
df_train = df_train[required_columns]

# Data Preprocessing
label_encoder_activity = LabelEncoder()
df_train['Activity'] = label_encoder_activity.fit_transform(df_train['Activity'])
activity_mapping = dict(zip(label_encoder_activity.classes_, range(len(label_encoder_activity.classes_))))
print("Activity Mapping:", activity_mapping)

# Separate features and targets
X_train = df_train.drop(['Activity'], axis=1)
y_train_activity = df_train['Activity']

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Upload the test dataset
print("\nUpload the test dataset:")
uploaded_test = files.upload()

# Read the test dataset
df_test = pd.read_csv(io.BytesIO(list(uploaded_test.values())[0]))

# Select specified columns for testing
for col in required_columns:
    if col not in df_test.columns:
        raise ValueError(f"Missing column in test dataset: {col}")
df_test = df_test[required_columns]

# Data Preprocessing for the test set
df_test['Activity'] = label_encoder_activity.transform(df_test['Activity'])

# Separate features and targets for the test set
X_test = df_test.drop(['Activity'], axis=1)
y_test_activity = df_test['Activity']

# Standardize numerical features for the test set
X_test_scaled = scaler.transform(X_test)

# Define a model builder for hyperparameter tuning
def model_builder(hp):
    model = Sequential()
    # Tune number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    model.add(Dense(hp_units, activation='relu', input_dim=X_train_scaled.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(label_encoder_activity.classes_), activation='softmax', name='activity_output'))
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Hyperparameter tuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train_activity, epochs=20, validation_split=0.2)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Train the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train_activity, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the testing set
y_pred_activity = model.predict(X_test_scaled)
y_pred_activity = y_pred_activity.argmax(axis=1)

# Save the test set with predictions to a CSV file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'test_set_with_predictions_{timestamp}.csv'
df_test['Predicted Activity'] = label_encoder_activity.inverse_transform(y_pred_activity)
df_test.to_csv(output_file, index=False)

# Download the CSV file with predictions
files.download(output_file)

# Print accuracy of Activity vs predicted activity
accuracy_activity = sum(y_test_activity == y_pred_activity) / len(y_test_activity)
print(f'Accuracy of Activity vs Predicted Activity: {accuracy_activity * 100:.2f}%')


GPU available: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Upload the training dataset:


Saving updated_output_merged (1) (4).csv to updated_output_merged (1) (4) (2).csv
Saving updated_output_merged (2) (4).csv to updated_output_merged (2) (4) (2).csv
Saving updated_output_merged (3) (4).csv to updated_output_merged (3) (4) (2).csv
Saving updated_output_merged (4) (4).csv to updated_output_merged (4) (4) (2).csv
Saving updated_output_merged (5) (4).csv to updated_output_merged (5) (4) (2).csv
Saving updated_output_merged (7) (4).csv to updated_output_merged (7) (4) (2).csv
Saving updated_output_merged (8) (4).csv to updated_output_merged (8) (4) (2).csv
Saving updated_output_merged (9) (4).csv to updated_output_merged (9) (4) (2).csv
Saving updated_output_merged (10) (4).csv to updated_output_merged (10) (4) (2).csv
Saving updated_output_merged (11) (5).csv to updated_output_merged (11) (5) (2).csv
Saving updated_output_merged (12) (5).csv to updated_output_merged (12) (5) (2).csv
Activity Mapping: {'Ball Throwing': 0, 'No Activity': 1, 'Paused': 2, 'Talking': 3, 'Walking

Saving updated_output_merged (6) (4).csv to updated_output_merged (6) (4) (2).csv
Reloading Tuner from my_dir/intro_to_kt/tuner0.json
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.5841 - loss: 1.2176 - val_accuracy: 0.7011 - val_loss: 1.6353
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8186 - loss: 0.5105 - val_accuracy: 0.7816 - val_loss: 0.5754
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8961 - loss: 0.3275 - val_accuracy: 0.7011 - val_loss: 0.7807
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9126 - loss: 0.2672 - val_accuracy: 0.9080 - val_loss: 0.2169
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9207 - loss: 0.2692 - val_accuracy: 0.9425 - val_loss: 0.1371
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9463 - loss: 0.1818 - val_accuracy: 0.9540 - val_loss: 0.1219
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9512 - loss: 0.1307 - val_accuracy: 0.9655 - val_loss: 0.1042
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9569 - loss: 0.1271 - val_accuracy: 0.9425 - val_loss: 0.1396

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Accuracy of Activity vs Predicted Activity: 67.91%
